In [ ]:
!nvidia-smi

Thu Aug 14 15:59:52 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Install libraries we’ll use
!pip install -q ultralytics opencv-python onnx onnxruntime-gpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
!which trtexec

In [ ]:
!trtexec --version

/bin/bash: line 1: trtexec: command not found


In [ ]:
!pip install tensorrt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 67.3 MB/s eta 0:00:00
  Created wheel for tensorrt: filename=tensorrt-10.13.2.6-py2.py3-none-any.whl size=46436 sha256=73db0168556a2f30a1a67b1aeac0c6474ec7e99a4e625026326cf1aebcf2af91
  Stored in directory: /root/.cache/pip/wheels/c6/c3/4f/b5a62d55edf240d6a6e82bdba1994887a4c861d7a1ed415d0d
  Created wheel for tensorrt_cu13: filename=tensorrt_cu13-10.13.2.6-py2.py3-none-any.whl size=17437 sha256=4c0b4c4ed17ed10118b6fd7586cf572de6c09562054aa2893c4a65a000372720
  Stored in directory: /root/.cache/pip/wheels/25/74/87/c19c3dec6eb4d121b960b20e01306caea68b42a6bd82279a2e
  Created wheel for tensorrt_cu13_libs: filename=tensorrt_cu13_libs-10.13.2.6-

In [1]:
import torch, os, sys, pathlib
assert torch.cuda.is_available(), "Colab GPU is not enabled. Go to Runtime → Change runtime type → GPU."

from google.colab import drive
drive.mount('/content/drive', force_remount=True)



Mounted at /content/drive


In [2]:
VIDEO_PATH = "/content/drive/MyDrive/yolo11_trt_demo/video/sample_video.mp4"

from pathlib import Path
vp = Path(VIDEO_PATH)
assert vp.exists(), f"Video not found at: {VIDEO_PATH}\nDouble-check the path under /content/drive/MyDrive/..."

print("Video OK:", VIDEO_PATH)

Video OK: /content/drive/MyDrive/yolo11_trt_demo/video/sample_video.mp4


In [4]:
import sys, platform, subprocess, os

!nvidia-smi
print("Python:", sys.version)
print("Platform:", platform.platform())


Fri Aug 15 10:11:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import sys, subprocess, importlib

def pip(*args):
    cmd = [sys.executable, "-m", "pip"] + list(args)
    print(">>>", " ".join(cmd))
    subprocess.check_call(cmd)

# 1) Try the standard metapackage (from NVIDIA’s index)
ok = False
try:
    import tensorrt as trt
    print("TensorRT already present:", trt.__version__)
    ok = True
except Exception as e:
    print("TensorRT not importable yet:", e)

if not ok:
    try:
        pip("install", "-U", "--extra-index-url", "https://pypi.nvidia.com", "tensorrt")
        import tensorrt as trt
        print("Installed TensorRT:", trt.__version__)
        ok = True
    except Exception as e:
        print("Standard install failed:", e)

# 2) Fallback: explicitly install the CUDA 12 build (common on Colab)
if not ok:
    try:
        pip("install", "-U", "--extra-index-url", "https://pypi.nvidia.com", "tensorrt-cu12")
        import tensorrt as trt
        print("Installed TensorRT (cu12):", trt.__version__)
        ok = True
    except Exception as e:
        print("cu12 install failed:", e)

# 3) Final nudge: force-reinstall (clears any partial/broken state)
if not ok:
    try:
        pip("install", "-U", "--force-reinstall", "--extra-index-url", "https://pypi.nvidia.com", "tensorrt")
        import tensorrt as trt
        print("Installed TensorRT (force):", trt.__version__)
        ok = True
    except Exception as e:
        print("Force install failed:", e)

assert ok, "TensorRT pip wheels were not installed. Check Python version compatibility in this Colab runtime."


TensorRT not importable yet: No module named 'tensorrt'
>>> /usr/bin/python3 -m pip install -U --extra-index-url https://pypi.nvidia.com tensorrt
Installed TensorRT: 10.13.2.6


In [3]:
# ——— Inspect runtime ———
import sys, platform, os, subprocess, re

print("Python:", sys.version)
print("Platform:", platform.platform())

# Colab GPU?
!nvidia-smi || echo "No GPU visible. Enable GPU: Runtime → Change runtime type → GPU"

# Quick Python version advice for TensorRT wheels:
py_major_minor = f"{sys.version_info.major}.{sys.version_info.minor}"
if py_major_minor not in {"3.10", "3.11"}:
    print(f"NOTE: You’re on Python {py_major_minor}. If TensorRT wheels fail to import, "
          "switch to Python 3.10 or 3.11 in Colab and rerun.")

def pip(*args):
    cmd = [sys.executable, "-m", "pip"] + list(args)
    print(">>>", " ".join(cmd))
    subprocess.check_call(cmd)

# ——— Clean up any partial/broken installs (optional but helps) ———
try:
    pip("uninstall", "-y", "tensorrt", "tensorrt-cu12")
except Exception as e:
    print("Uninstall note:", e)

# ——— Install core deps ———
pip("install", "-U", "ultralytics", "onnx", "onnxruntime-gpu")

# Try TRT cu12 first (Colab GPUs are CUDA 12.x), then fallback to meta
trt_ok = False
for pkg in ( "tensorrt-cu12", "tensorrt" ):
    try:
        pip("install", "-U", "--extra-index-url", "https://pypi.nvidia.com", pkg)
        import importlib; importlib.invalidate_caches()
        import tensorrt as trt  # test import right after install
        print("TensorRT imported OK:", trt.__version__, f"({pkg})")
        trt_ok = True
        break
    except Exception as e:
        print(f"Install/import failed for {pkg} →", e)

if not trt_ok:
    raise SystemExit("TensorRT wheel install failed. If you’re on Python 3.12, switch runtime to 3.10/3.11 and rerun.")


Python: 3.11.13 (main, Jun  4 2025, 08:57:29) [GCC 11.4.0]
Platform: Linux-6.1.123+-x86_64-with-glibc2.35
Fri Aug 15 10:15:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                 

In [4]:
try:
    import ultralytics, onnx, tensorrt as trt
    import cv2, numpy as np, time, yaml, glob, shutil, math, random
    print("Ultralytics:", ultralytics.__version__)
    print("TensorRT:", trt.__version__)
    print("OpenCV:", cv2.__version__)
except Exception as e:
    print("Imports still failing →", repr(e))
    print("\nFixes:")
    print("1) Runtime → Restart runtime (one time) and re-run the two cells above.")
    print("2) Ensure Colab is set to GPU and Python 3.10/3.11.")
    raise

print("\n✅ Environment is ready. Proceed with the next cells (Config → Calibration → Exports → Benchmark).")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.179
TensorRT: 10.13.2.6
OpenCV: 4.12.0

✅ Environment is ready. Proceed with the next cells (Config → Calibration → Exports → Benchmark).


In [ ]:
from ultralytics import YOLO
from pathlib import Path

# Inference & export 
IMGSZ = 640
CONF  = 0.50
WARMUP_FRAMES = 20    
LIMIT_FRAMES  = None   

# Folders
OUT_DIR   = Path("/content/outs"); OUT_DIR.mkdir(parents=True, exist_ok=True)
CALIB_DIR = Path("/content/calib/images"); CALIB_DIR.mkdir(parents=True, exist_ok=True)
CALIB_YAML = Path("/content/calib.yaml")

# Outputs
ENG_FP32 = OUT_DIR/"yolo11m_fp32.engine"
ENG_FP16 = OUT_DIR/"yolo11m_fp16.engine"
ENG_INT8 = OUT_DIR/"yolo11m_int8.engine"

# Baseline model
BASE_PT = YOLO("yolo11m.pt")


In [ ]:

from pathlib import Path
import os, cv2, random
VIDEO_PATH = "/content/drive/MyDrive/yolo11_trt_demo/video/sample_video.mp4"  
assert Path(VIDEO_PATH).is_file(), f"Video not found: {VIDEO_PATH}"

# Quick try: can OpenCV open it?
_cap = cv2.VideoCapture(VIDEO_PATH)
if not _cap.isOpened():
    print("OpenCV couldn't open the original file. Trying a transcode to a Colab-friendly MP4...")
    !ffmpeg -y -i "$VIDEO_PATH" -c:v libx264 -pix_fmt yuv420p -movflags +faststart /content/_video_fixed.mp4
    VIDEO_PATH = "/content/_video_fixed.mp4"
    print("Transcoded →", VIDEO_PATH)
_cap.release()

def extract_calib_frames_from_videos(video_paths, out_dir, target_total=480, imgsz=640, seed=0):
    from pathlib import Path
    import cv2, numpy as np

   
    if isinstance(video_paths, (str, Path)):
        video_paths = [str(video_paths)]
    else:
        video_paths = [str(p) for p in video_paths]

    random.seed(seed)
    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    nvid = max(1, len(video_paths))
    per_video = max(1, target_total // nvid)
    saved = 0

    for vp in video_paths:
        cap = cv2.VideoCapture(vp)
        assert cap.isOpened(), f"Cannot open video: {vp}"
        n = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) or per_video*2

        
        if per_video == 1:
            idxs = [n // 2]
        else:
            idxs = sorted(set(int(i*(n-1)/(per_video-1)) for i in range(per_video)))
        jitter = max(1, n // (per_video*8 if per_video>0 else 8))
        idxs = [max(0, min(n-1, i + random.randint(-jitter, jitter))) for i in idxs]

        for idx in idxs:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ok, frame = cap.read()
            if not ok:
                continue
            h, w = frame.shape[:2]
            side = min(h, w)
            y0 = (h - side) // 2; x0 = (w - side) // 2
            crop = frame[y0:y0+side, x0:x0+side]
            img = cv2.resize(crop, (imgsz, imgsz), interpolation=cv2.INTER_AREA)
            cv2.imwrite(str(out_dir / f"calib_{saved:05d}.jpg"), img)
            saved += 1
        cap.release()
    print(f"Saved {saved} calibration frames → {out_dir}")
    return saved


CALIB_DIR = "/content/calib/images"
IMGSZ = 640
num_calib = extract_calib_frames_from_videos([VIDEO_PATH], CALIB_DIR, target_total=480, imgsz=IMGSZ)
print("num_calib =", num_calib)



Saved 480 calibration frames → /content/calib/images
num_calib = 480


In [ ]:

model = BASE_PT
names = getattr(getattr(model, "model", None), "names", None) or getattr(model, "names", None)
if isinstance(names, dict):
    names = [names[i] for i in range(len(names))]
if not isinstance(names, (list, tuple)):
    names = [str(i) for i in range(80)] 

calib_cfg = {
    "path": "/content/calib",
    "train": "images",     
    "val": "images",       # calibrator reads these for INT8 activation ranges
    "nc": int(len(names)),
    "names": list(map(str, names))
}
with open(CALIB_YAML, "w") as f:
    yaml.safe_dump(calib_cfg, f)

print(f"Calibration YAML → {CALIB_YAML}\nClasses: {len(names)}")


Calibration YAML → /content/calib.yaml
Classes: 80


In [ ]:
def find_newest_engine(since_time: float):
    paths = [p for p in glob.glob("**/*.engine", recursive=True)]
    paths = [(p, os.path.getmtime(p)) for p in paths if os.path.getmtime(p) >= since_time - 5]
    if not paths:
        
        paths = [(p, os.path.getmtime(p)) for p in glob.glob("**/*.engine", recursive=True)]
    if not paths:
        return None
    paths.sort(key=lambda x: x[1], reverse=True)
    return paths[0][0]

def export_engine(dest_path: Path, *, half=False, int8=False, data_yaml=None, imgsz=640, batch=1, workspace=4.0, fraction=None):
    t0 = time.time()
    args = dict(format="engine", imgsz=imgsz, half=half, int8=int8, batch=batch, workspace=workspace)
    if int8:
        assert data_yaml is not None and Path(data_yaml).exists(), "INT8 export needs data=calibration_yaml"
        args.update(dict(data=str(data_yaml)))
        if fraction is not None:
            args.update(dict(fraction=float(fraction)))
    # Do export
    _ = model.export(**args)
    # Locate the engine file produced
    found = find_newest_engine(t0)
    assert found, "Engine file not found after export."
    shutil.move(found, dest_path)
    print(("FP16" if half else "INT8" if int8 else "FP32"), "engine →", dest_path)
    return str(dest_path)


In [16]:
# FP32
export_engine(ENG_FP32, half=False, int8=False, imgsz=IMGSZ, batch=1, workspace=4.0)

# FP16
export_engine(ENG_FP16, half=True,  int8=False, imgsz=IMGSZ, batch=1, workspace=4.0)

# INT8  (use all calibration images; you can lower 'fraction' to speed up)
calib_batch = min(8, max(1, num_calib // 8))
export_engine(ENG_INT8, half=False, int8=True, data_yaml=CALIB_YAML, imgsz=IMGSZ, batch=calib_batch, workspace=4.0, fraction=1.0)


WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
💡 ProTip: Export to OpenVINO format for best performance on Intel hardware. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLO11m summary (fused): 125 layers, 20,091,712 parameters, 0 gradients, 68.0 GFLOPs

PyTorch: starting from 'yolo11m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (38.8 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.59'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 2.9s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.18.0 opset 19...
ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 8.5s, saved as 'yolo11m.onnx' (76.9 MB)

TensorRT: starting export with TensorRT 10.13.2.6...
TensorRT: input "images" with shape(1, 3,

Fast image access ✅ (ping: 0.0±0.0 ms, read: 94.0±134.2 MB/s, size: 80.0 KB)



Scanning /content/calib/labels... 0 images, 480 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<00:00, 669.22it/s]

WARNING ⚠️ No labels found in /content/calib/labels.cache. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


New cache created: /content/calib/labels.cache
WARNING ⚠️ Labels are missing or empty in /content/calib/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.
TensorRT: input "images" with shape(8, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(8, 84, 8400) DataType.FLOAT
TensorRT: building INT8 engine as yolo11m.engine
TensorRT: export success ✅ 599.9s, saved as 'yolo11m.engine' (25.0 MB)

Export complete (600.7s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo11m.engine imgsz=640 int8 
Validate:        yolo val task=detect model=yolo11m.engine imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml int8 
Visualize:       https://netron.app
INT8 engine → /content/outs/yolo11m_int8.engine


'/content/outs/yolo11m_int8.engine'

In [18]:
# Re-export INT8 with batch=1 so it matches single-frame inference
from pathlib import Path
import os, time, shutil, glob

def find_newest_engine(since_time: float):
    paths = [p for p in glob.glob("**/*.engine", recursive=True)]
    paths = [(p, os.path.getmtime(p)) for p in paths if os.path.getmtime(p) >= since_time - 5]
    if not paths:
        paths = [(p, os.path.getmtime(p)) for p in glob.glob("**/*.engine", recursive=True)]
    if not paths:
        return None
    paths.sort(key=lambda x: x[1], reverse=True)
    return paths[0][0]

def export_engine(dest_path, *, half=False, int8=False, data_yaml=None, imgsz=640, batch=1, workspace=4.0, fraction=None, dynamic=False):
    t0 = time.time()
    args = dict(format="engine", imgsz=imgsz, half=half, int8=int8, batch=batch, workspace=workspace, dynamic=dynamic)
    if int8:
        assert data_yaml is not None and Path(data_yaml).exists(), "INT8 export needs data=calibration_yaml"
        args.update(dict(data=str(data_yaml)))
        if fraction is not None:
            args.update(dict(fraction=float(fraction)))
    _ = model.export(**args)
    found = find_newest_engine(t0)
    assert found, "Engine file not found after export."
    shutil.move(found, dest_path)
    print(("FP16" if half else "INT8" if int8 else "FP32"), "engine →", dest_path)

# Remove old INT8 engine (batch=8) if present
try:
    Path(ENG_INT8).unlink()
except FileNotFoundError:
    pass

# Rebuild INT8 with batch=1 (static). Keep fraction=1.0; you can use 0.25 for a quick test build.
export_engine(ENG_INT8, half=False, int8=True, data_yaml=CALIB_YAML,
              imgsz=IMGSZ, batch=1, workspace=4.0, fraction=1.0, dynamic=False)


WARNING ⚠️ TensorRT requires GPU export, automatically assigning device=0
Ultralytics 8.3.179 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

PyTorch: starting from 'yolo11m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (38.8 MB)

ONNX: starting export with onnx 1.18.0 opset 19...
ONNX: slimming with onnxslim 0.1.64...
ONNX: export success ✅ 2.7s, saved as 'yolo11m.onnx' (76.9 MB)

TensorRT: starting export with TensorRT 10.13.2.6...
TensorRT: collecting INT8 calibration images from 'data=/content/calib.yaml'
Fast image access ✅ (ping: 0.0±0.0 ms, read: 1171.6±504.8 MB/s, size: 79.3 KB)


Scanning /content/calib/labels.cache... 0 images, 480 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<?, ?it/s]

WARNING ⚠️ Labels are missing or empty in /content/calib/labels.cache, training may not work correctly. See https://docs.ultralytics.com/datasets for dataset formatting guidance.


TensorRT: input "images" with shape(1, 3, 640, 640) DataType.FLOAT
TensorRT: output "output0" with shape(1, 84, 8400) DataType.FLOAT
TensorRT: building INT8 engine as yolo11m.engine
TensorRT: export success ✅ 286.3s, saved as 'yolo11m.engine' (24.7 MB)

Export complete (286.4s)
Results saved to /content
Predict:         yolo predict task=detect model=yolo11m.engine imgsz=640 int8 
Validate:        yolo val task=detect model=yolo11m.engine imgsz=640 data=/ultralytics/ultralytics/cfg/datasets/coco.yaml int8 
Visualize:       https://netron.app
INT8 engine → /content/outs/yolo11m_int8.engine


In [19]:
from ultralytics import YOLO
import cv2, time, numpy as np

def benchmark_and_write(model_path, tag, out_mp4, imgsz=IMGSZ, conf=CONF,
                        warmup=WARMUP_FRAMES, limit=LIMIT_FRAMES, device=0):
    is_engine = str(model_path).endswith(".engine")
    model = YOLO(model_path, task="detect") if is_engine else YOLO(model_path)

    cap = cv2.VideoCapture(str(VIDEO_PATH))
    assert cap.isOpened(), f"Cannot open video: {VIDEO_PATH}"
    w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps_in = cap.get(cv2.CAP_PROP_FPS) or 25.0

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    writer = cv2.VideoWriter(str(out_mp4), fourcc, fps_in, (w, h))

    times = []; i = 0
    while True:
        ok, frame = cap.read()
        if not ok: break
        t0 = time.time()
        res = model.predict(source=frame, imgsz=imgsz, conf=conf, device=device, verbose=False)
        dt = time.time() - t0
        if i >= warmup: times.append(dt)
        plotted = res[0].plot()
        cv2.putText(plotted, f"{tag}", (22, 42), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,0), 4, cv2.LINE_AA)
        cv2.putText(plotted, f"{tag}", (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255,255,255), 2, cv2.LINE_AA)
        writer.write(plotted)
        i += 1
        if limit and i >= limit: break

    cap.release(); writer.release()
    mean = float(np.mean(times)) if times else float("nan")
    fps  = (1.0/mean) if np.isfinite(mean) else 0.0
    return mean, fps, len(times)


In [ ]:

try:
    results
except NameError:
    results = [] 

mean8, fps8, n8 = benchmark_and_write(ENG_INT8, "TensorRT INT8", OUT_DIR/"trt_int8.mp4")
results.append(("TensorRT INT8", mean8, fps8, n8))

print("\n=== Inference Speed Summary (ignoring first", WARMUP_FRAMES, "frames) ===")
print(f"{'Model':18}  {'avg ms/frame':>14}  {'FPS':>9}  {'timed frames':>13}")
for name, mean, fps, n in results:
    ms = mean*1000 if np.isfinite(mean) else float('nan')
    print(f"{name:18}  {ms:14.2f}  {fps:9.2f}  {n:13d}")


Loading /content/outs/yolo11m_int8.engine for TensorRT inference...

=== Inference Speed Summary (ignoring first 20 frames) ===
Model                 avg ms/frame        FPS   timed frames
PyTorch FP32                 27.30      36.63            669
TensorRT FP32                30.39      32.91            669
TensorRT FP16                17.39      57.50            669
TensorRT INT8                14.32      69.84            669


In [21]:
# Mount (safe to re-run)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
import shutil, os

SRC = Path("/content/outs")
DST = Path("/content/drive/MyDrive/yolo11_trt_outs")
DST.mkdir(parents=True, exist_ok=True)

# Copy engines and videos
copied = 0
for p in SRC.glob("*"):
    if p.suffix.lower() in {".engine", ".mp4"}:
        shutil.copy2(p, DST / p.name)
        copied += 1

print(f"Copied {copied} files → {DST}")

# List what’s in Drive with sizes
for p in sorted(DST.glob("*")):
    sz = os.path.getsize(p) / (1024*1024)
    print(f"{p.name:30}  {sz:6.1f} MB")


Mounted at /content/drive
Copied 7 files → /content/drive/MyDrive/yolo11_trt_outs
baseline_pytorch_fp32.mp4         42.9 MB
trt_fp16.mp4                      43.2 MB
trt_fp32.mp4                      43.1 MB
trt_int8.mp4                      46.9 MB
yolo11m_fp16.engine               41.9 MB
yolo11m_fp32.engine               86.6 MB
yolo11m_int8.engine               24.7 MB


In [ ]:
from pathlib import Path
import shutil, os

SRC_IMG  = Path("/content/calib/images")
DST_CAL  = Path("/content/drive/MyDrive/yolo11_trt_outs/calib/images")
DST_CAL.mkdir(parents=True, exist_ok=True)

count = 0
for p in SRC_IMG.glob("*.jpg"):
    shutil.copy2(p, DST_CAL / p.name)
    count += 1


SRC_YAML = Path("/content/calib.yaml")
if SRC_YAML.exists():
    shutil.copy2(SRC_YAML, DST_CAL.parent / "calib.yaml")

print(f"Copied {count} calibration images to {DST_CAL}")


Copied 480 calibration images to /content/drive/MyDrive/yolo11_trt_outs/calib/images
